In [1]:
from transformers import AutoModel, AutoTokenizer
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from qdrant_client import QdrantClient
from dotenv import dotenv_values
import yaml

DBPATH = "data/db/"
with open("config.yml") as yml:
    conf = yaml.safe_load(yml)
ENV = dotenv_values(".env")
encoder = "sts/paraphrase-multilingual-MiniLM-L12-v2"

c:\Users\hp\AppData\Local\pypoetry\Cache\virtualenvs\llm-data-generator-vX9nrdqr-py3.10\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
encoder_tokenizer = AutoTokenizer.from_pretrained(encoder)
encoder_model = AutoModel.from_pretrained(encoder)

In [3]:
client = QdrantClient(url = ENV["QDRANT_URL"], location=DBPATH, timeout=50)

In [35]:
# first search
query = 'كرة القدم'
inputs = encoder_tokenizer(
    text=query,
    padding=True,
    truncation=True,
    return_tensors="pt",
    max_length=conf["vector_size"]
)
results = encoder_model(**inputs)
embeddings = results.last_hidden_state[:, 0, :].cpu().detach().numpy()
embeddings = embeddings.flatten().tolist()

query_results = client.search(
    collection_name="prod",
    query_vector=embeddings,
    with_payload=["description"],
    limit=3
)

In [36]:
query_results

[ScoredPoint(id='44620bd9-e266-c639-e5a0-b81645f5a941', version=14, score=0.62364066, payload={'description': 'انطلق أول مران لفريق الكرة بالنادي الأهلي منذ قليل ملعب جامعة بريتوريا، استعدادا لمباراة صن داونز التي تقام في الثالثة من عصر بعد غد'}, vector=None),
 ScoredPoint(id='f0b6282b-6f0c-20d5-6c60-a8d1510dbfc7', version=11, score=0.6233935, payload={'description': 'في كل عام يتنافس أفضل لاعبى كرة القدم للحصول على واحدة من الجوائز الفردية الأبرز وهي الكرة الذهبية، التي تمنحها مجلة فرانس فوتبول فى حفل سنوي.'}, vector=None),
 ScoredPoint(id='d4fdd71d-69b3-8b2f-4e65-75e527c45e78', version=2, score=0.6089852, payload={'description': 'أكد...'}, vector=None)]

In [37]:
prompt_template = """
Context:
{context}

Instructions:
Answer the following Question Using the Context provided: {question}

---

[User prompt or input for the model goes here.]

"""
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

In [42]:
context = query_results[0].payload['description']
question = "ما هى أخر أخبار الرياضة؟"

In [43]:
context

'انطلق أول مران لفريق الكرة بالنادي الأهلي منذ قليل ملعب جامعة بريتوريا، استعدادا لمباراة صن داونز التي تقام في الثالثة من عصر بعد غد'

In [27]:
llm = OpenAI(openai_api_key=ENV['OPENAI_API'], model_name = "gpt-3.5-turbo", temperature = 0.7)

c:\Users\hp\AppData\Local\pypoetry\Cache\virtualenvs\llm-data-generator-vX9nrdqr-py3.10\lib\site-packages\langchain\llms\openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
c:\Users\hp\AppData\Local\pypoetry\Cache\virtualenvs\llm-data-generator-vX9nrdqr-py3.10\lib\site-packages\langchain\llms\openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [44]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [45]:
llm_chain.run({'context':context, 'question':question})

'أخر أخبار الرياضة هي أن فريق الكرة بالنادي الأهلي بدأ أول مران له في ملعب جامعة بريتوريا استعداداً لمباراة صن داونز التي ستقام بعد غد.'

In [46]:
# first search
query = 'الذهب'
inputs = encoder_tokenizer(
    text=query,
    padding=True,
    truncation=True,
    return_tensors="pt",
    max_length=conf["vector_size"]
)
results = encoder_model(**inputs)
embeddings = results.last_hidden_state[:, 0, :].cpu().detach().numpy()
embeddings = embeddings.flatten().tolist()

query_results = client.search(
    collection_name="prod",
    query_vector=embeddings,
    with_payload=["description"],
    limit=3
)

In [47]:
query_results

[ScoredPoint(id='1237d46d-e08f-0a9e-ca5c-1e5e317df123', version=12, score=0.6420081, payload={'description': 'تابعة لاتحاد الغرف التجارية المصرية، وصل سعر جرام الذهب من عيار 24 إلى 2950 جنيهاً، بينما وصل سعر …'}, vector=None),
 ScoredPoint(id='d3d21030-5836-7009-ceec-70fad6b8126d', version=2, score=0.6121622, payload={'description': 'شهدت أسعار الذهب في مصر تراجع ملحوظ بحوالي 20 جنيها للجرام، وذلك بعد ساعات محدودة من صدور قرار مجلس الوزراء بالسماح بدخول الذهب بدون جمارك والذي'}, vector=None),
 ScoredPoint(id='e9528e71-44a5-d4a1-7367-cc643fc6eff7', version=9, score=0.59381783, payload={'description': 'تشهد أسعار الذهب في البورصة العالمية انخفاضات منذ صباح اليوم وذلك بالتزامن مع اتجاه الاحتياطي الفيدرالي الأمريكي لتثبيت الفائدة عند مستوياتها 5.25 و 5.50 %،'}, vector=None)]

In [48]:
context = query_results[0].payload['description']
question = "ما هى أسعار الذهب الأن؟"

In [49]:
llm_chain.run({'context':context, 'question':question})

'وصل سعر جرام الذهب من عيار 24 إلى 2950 جنيهاً.'

In [54]:
# first search
query = 'محمد صلاح'
inputs = encoder_tokenizer(
    text=query,
    padding=True,
    truncation=True,
    return_tensors="pt",
    max_length=conf["vector_size"]
)
results = encoder_model(**inputs)
embeddings = results.last_hidden_state[:, 0, :].cpu().detach().numpy()
embeddings = embeddings.flatten().tolist()

query_results = client.search(
    collection_name="prod",
    query_vector=embeddings,
    with_payload=["description"],
    limit=3
)

In [59]:
context = query_results[1].payload['description']
question = "ما هى أخر أخبار محمد صلاح مع ليفربول؟"

In [60]:
llm_chain.run({'context':context, 'question':question})

'أخر أخبار محمد صلاح مع ليفربول هي أنه ينافس على جائزة أفضل لاعب من رابطة اللاعبين المحترفين في الدوري الإنجليزي عن شهر أكتوبر الماضي.'